# Installing Hub

In [ ]:
!pip3 install hub --quiet
import os 
os.kill(os.getpid(), 9) 

     |████████████████████████████████| 261 kB 8.6 MB/s 
     |████████████████████████████████| 131 kB 60.1 MB/s 
     |████████████████████████████████| 1.8 MB 47.2 MB/s 
     |████████████████████████████████| 81 kB 7.0 MB/s 
     |████████████████████████████████| 5.8 MB 54.3 MB/s 
     |████████████████████████████████| 105 kB 46.6 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 551 kB 73.0 MB/s 
     |████████████████████████████████| 256 kB 73.4 MB/s 
     |████████████████████████████████| 152 kB 67.6 MB/s 
     |████████████████████████████████| 3.0 MB 43.2 MB/s 
     |████████████████████████████████| 75 kB 4.5 MB/s 
     |████████████████████████████████| 111 kB 47.1 MB/s 
     |████████████████████████████████| 8.5 MB 43.4 MB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 127 kB 74.6 MB/s 
     |████████████████████████████████| 41 kB 412 kB/s 
     |███████████████████

# Download raw dataset

In [1]:
from IPython.display import clear_output

In [ ]:
# Download Dataset here
!wget https://github.com/karoldvl/ESC-50/archive/master.zip
!unzip master.zip
clear_output()

In [35]:
import hub
from PIL import Image 
import numpy as np 
import os
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
import time
from tqdm import tqdm

In [6]:
#@title Login to ActiveLoop

%env BUGGER_OFF=True
!activeloop login -u username -p password
!activeloop reporting --off

env: BUGGER_OFF=True
Successfully logged in to Activeloop.


# Creating dataset on hub

**Activeloop API** : https://docs.activeloop.ai/api-basics

In [7]:
!ls

ActiveloopTemplate.ipynb  ESC-50-master  master.zip


In [8]:
basepath = 'ESC-50-master'

In [9]:
df = pd.read_csv(os.path.join(basepath, 'meta', 'esc50.csv'))

In [11]:
df.iloc[0]

filename    1-100032-A-0.wav
fold                       1
target                     0
category                 dog
esc10                   True
src_file              100032
take                       A
Name: 0, dtype: object

In [19]:
class_names = df.category.unique().tolist()

In [37]:
# hubname = './esc50-hub'
hubname = 'hub://activeloop/esc50'
hub.delete(hubname, large_ok=True)
ds = hub.dataset(hubname)

start = time.time()

with ds:
    ds.create_tensor('audio', htype='audio', sample_compression='wav')
    ds.create_tensor('labels', htype='class_label', class_names=class_names)
    ds.create_tensor('target')
    ds.create_tensor('fold')
    ds.create_tensor('esc10')
    ds.create_tensor('src_file', htype='text')
    ds.create_tensor('take', htype='text')
    
    audio_folder = os.path.join(basepath, 'audio')
    
    for index, row in tqdm(df.iterrows()):
        # print(f"[{index}] | {row['filename']}")
        audio_path = os.path.join(audio_folder, row['filename'])
        
        ds.append({
            'audio' : hub.read(audio_path),
            'labels' : class_names.index(row['category']),
            'target' : row['target'],
            'fold' : row['fold'],
            'esc10' : row['esc10'],
            'src_file' : str(row['src_file']),
            'take' : row['take']
        })
        
stop = time.time()
print(f'Time elapsed : {stop - start}')

Your Hub dataset has been successfully created!
The dataset is private so make sure you are logged in!
This dataset can be visualized at https://app.activeloop.ai/activeloop/esc50.


2000it [23:04,  1.44it/s]


Time elapsed : 1899.3558781147003
